# Modeling Agricultural Variables
## Python modules

In [1]:
import warnings
import time
import os

import dask
from dask.distributed import Client
import concurrent.futures

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import geopandas as gpd
from shapely.geometry import Polygon
import pyarrow
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from scipy.stats import spearmanr
from scipy.linalg import LinAlgWarning
from scipy.stats import pearsonr
from hilbertcurve.hilbertcurve import HilbertCurve

import math
import seaborn as sns

This notebook will be used to prepare our feature and ground-truth data for our modeling process. At this point, we have a directory of .feather files that contain our features (see this notebook for feature generation). We want to concatentate these together, and perform the necessary operations to achieve a dataframe where a row is one observation unit; its columns are the associated features and the ground truth data that we want to train the model on. Broadly, we want our dataframe to take on the following form:

| observation_unit |  outcome_1 | outcome_2 | feature_1| feature_2 | feature_3
| ----|  ---- | ---- | -- | -- | -- |
| 1   |  $y_{1}$ | $y_{2}$ | $x_{1}$ | $x_{2}$ | $x_{3}$
| 2   |  $y_{1}$ | $y_{2}$ | $x_{1}$| $x_{2}$ | $x_{3}$

Where we can run regress `outcome_1` and `outcome_2` on our features, using the following form:

$y_{1}$ = $\beta_{1}$$x_{1}$ + $\beta_{2}$$x_{2}$ + $\beta_{3}$$x_{3}$ + $\beta_{n}$$x_{n}$ 

In our case, an observation is one SEA/year. We have ground truth data that summarizes the total crop yield, total crop loss, as well as other pertinent agricultural variables. We want our dataframe described above to look like this:

| SEA | Year | Observed Yield (Tonnes) | Observed Loss (Tonnes) | feature1| feature2 | feature3
| ----| ----| ---- | ---- | -- | -- | -- |
| 1   | 2016 | 72 | 13 | 1.23 | 3.25 | 0.123
| 2   | 2016  | 50 | 7.5 | 0.78| 1.2 | 2.4

## Mild Preprocessing
### 1. Load in the data

First, we load in the feature data. This data was aggregated in the ___ notebook

In [2]:
# Let's read in the new concatenated features:
features = gpd.read_feather("/capstone/mosaiks/repos/modeling/data/sentinel_rgb_features_sea_save_2023_04_24.feather")


In [3]:
features.head()

,0,1,2,3,4,5,6,7,8,9,...,995,996,997,998,999,lon,lat,year,month,geometry
0,0.001058,0.000000,0.005181,1.014728,0.098902,0.000000,0.033511,1.810280,0.0,0.000000,...,3.478022,4.774719,0.011104,0.840888,0.000045,27.47466,-16.339357,2015,7,POINT (27.47466 -16.33936)
1,0.002010,0.000022,0.003418,1.058190,0.125574,0.000291,0.039863,1.858072,0.0,0.000029,...,3.526972,4.882292,0.024313,0.873838,0.000047,27.46466,-16.339357,2015,7,POINT (27.46466 -16.33936)
2,0.004124,0.000000,0.000000,1.069269,0.127892,0.000000,0.036240,1.871935,0.0,0.000054,...,3.552448,4.921965,0.022708,0.881511,0.000000,27.46466,-16.329357,2015,7,POINT (27.46466 -16.32936)
3,0.001559,0.000004,0.000000,1.103106,0.134511,0.000230,0.057822,1.941573,0.0,0.000000,...,3.571980,4.965928,0.021992,0.857326,0.000266,27.45466,-16.339357,2015,7,POINT (27.45466 -16.33936)
4,0.001731,0.000002,0.005510,1.056985,0.122743,0.000167,0.034343,1.860240,0.0,0.000000,...,3.535786,4.864718,0.009189,0.852267,0.000000,27.47466,-16.349357,2015,7,POINT (27.47466 -16.34936)


### Ground-Truth Data
Next, we read in our ground truth data, which was processed in the preprocessing notebook

In [4]:
# Name list because gpd doesn't read in column names correctly
cols = ["sea_unq", "year", "total_area_planted_ha", "total_area_harv_ha", "total_area_lost_ha", "total_harv_kg", "yield_kgha", "frac_area_harv", "frac_area_loss", "area_lost_fire", "maize", "groundnuts", "mixed_beans",  "popcorn", "sorghum", "soybeans", "sweet_potatoes", "bunding", "monocrop", "mixture", "frac_loss_drought",  "frac_loss_flood", "frac_loss_animal", "frac_loss_pests", "frac_loss_soil", "frac_loss_fert", "prop_till_plough",  "prop_till_ridge", "prop_notill", "prop_hand", "prop_mono", "prop_mix", "log_maize",  "log_sweetpotatoes", "log_groundnuts", "log_soybeans", "loss_ind", "drought_loss_ind", "flood_loss_ind", "animal_loss_ind", "pest_loss_ind", "geometry"]

# Read in the survey data
country_sea = gpd.read_file('/capstone/mosaiks/repos/preprocessing/data/ground_data_spatial/updated_data.shp')
country_sea.columns = cols
country_sea.head()

ERROR 1: PROJ: proj_create_from_database: Open of /Users/andrewbartnik/.conda/envs/mosaiks/share/proj failed


,sea_unq,year,total_area_planted_ha,total_area_harv_ha,total_area_lost_ha,total_harv_kg,yield_kgha,frac_area_harv,frac_area_loss,area_lost_fire,...,log_maize,log_sweetpotatoes,log_groundnuts,log_soybeans,loss_ind,drought_loss_ind,flood_loss_ind,animal_loss_ind,pest_loss_ind,geometry
0,1,2009.0,34.9725,34.9725,0.0000,57563.0,1645.950390,1.000000,0.000000,0.0,...,7.419992,8.455318,7.276696,7.107469,0.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
1,1,2010.0,32.2150,26.7625,5.4525,69925.0,2170.572715,0.830747,0.169253,0.0,...,7.479710,9.021598,8.309147,7.377759,1.0,0.0,0.0,0.0,1.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
2,1,2011.0,60.4075,59.7525,0.6550,120614.0,1996.672599,0.989157,0.010843,0.0,...,7.563515,8.759924,8.066608,-inf,0.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
3,1,2012.0,64.6175,57.6025,7.0150,105990.0,1640.267729,0.891438,0.108562,0.0,...,7.248387,8.836374,8.067501,6.476972,1.0,1.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
4,1,2013.0,325.5000,282.7500,42.7500,1975.0,6.067588,0.868664,0.131336,0.0,...,0.213193,-inf,4.264244,-inf,1.0,0.0,0.0,0.0,0.0,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."


We're going to make another object `sea_unq_join` which contains the spatial information and a unique key for each SEA. This will be handy later, when we need to join the features to the ground-truth data.

In [5]:
# Filter country_sea for unique values of 'seq_unq' and 'geometry'
sea_unq_join = country_sea[['sea_unq', 'geometry']].drop_duplicates()

# Display the filtered DataFrame
print(sea_unq_join)


      sea_unq                                           geometry
0           1  POLYGON ((27.82327 -13.65772, 27.82294 -13.657...
10          2  POLYGON ((27.99349 -13.46497, 27.99352 -13.464...
20          3  POLYGON ((28.09909 -13.51864, 28.09867 -13.516...
29          4  POLYGON ((28.31924 -13.42915, 28.31911 -13.426...
38          5  POLYGON ((28.39982 -13.51544, 28.40012 -13.514...
...       ...                                                ...
3571      388  POLYGON ((25.07771 -14.63920, 25.07732 -14.638...
3578      389  POLYGON ((22.74142 -14.00343, 22.73856 -14.002...
3585      390  POLYGON ((23.08604 -14.20026, 23.08957 -14.202...
3592      391  POLYGON ((24.36764 -16.62208, 24.36564 -16.621...
3599      392  POLYGON ((23.23962 -16.31204, 23.23876 -16.312...

[392 rows x 2 columns]


### 2. Organize the features by growing season



In [6]:
# Organize the features by growing season
# Carry months October, November, and December over to the following year's data
# These months represent the start of the growing season for the following year's maize yield
year_end = 2022

features['year'] = np.where(
    features['month'].isin([10, 11, 12]),
    features['year'] + 1, 
    features['year'])

features_gs = features[features['year'] <= year_end]

features_gs.sort_values(['year', 'month'], inplace=True)

/tmp/ipykernel_2493574/261356123.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_gs.sort_values(['year', 'month'], inplace=True)


### 3. Convert the features into a geo dataframe

We first create a GeoDataFrame (`features_new_gdf`) from the DataFrame features, using the longitude and latitude columns to create the geometry column (geospatial points) and specifying the CRS (Coordinate Reference System) as EPSG:4326.

In [7]:
# Create a geodataframe of the new features
features_new_gdf = gpd.GeoDataFrame(
    features_gs, 
    geometry = gpd.points_from_xy(x = features_gs.lon, y = features_gs.lat), 
    crs='EPSG:4326'
)

In [8]:
# Notes: Have to change the year, get an error rn. Also, check to make sure the number of cells is correct
features_new_gdf = features_new_gdf.replace([np.inf, -np.inf], np.nan)
features_new_gdf.head()

,0,1,2,3,4,5,6,7,8,9,...,995,996,997,998,999,lon,lat,year,month,geometry
0,0.001058,0.000000,0.005181,1.014728,0.098902,0.000000,0.033511,1.810280,0.0,0.000000,...,3.478022,4.774719,0.011104,0.840888,0.000045,27.47466,-16.339357,2015,7,POINT (27.47466 -16.33936)
1,0.002010,0.000022,0.003418,1.058190,0.125574,0.000291,0.039863,1.858072,0.0,0.000029,...,3.526972,4.882292,0.024313,0.873838,0.000047,27.46466,-16.339357,2015,7,POINT (27.46466 -16.33936)
2,0.004124,0.000000,0.000000,1.069269,0.127892,0.000000,0.036240,1.871935,0.0,0.000054,...,3.552448,4.921965,0.022708,0.881511,0.000000,27.46466,-16.329357,2015,7,POINT (27.46466 -16.32936)
3,0.001559,0.000004,0.000000,1.103106,0.134511,0.000230,0.057822,1.941573,0.0,0.000000,...,3.571980,4.965928,0.021992,0.857326,0.000266,27.45466,-16.339357,2015,7,POINT (27.45466 -16.33936)
4,0.001731,0.000002,0.005510,1.056985,0.122743,0.000167,0.034343,1.860240,0.0,0.000000,...,3.535786,4.864718,0.009189,0.852267,0.000000,27.47466,-16.349357,2015,7,POINT (27.47466 -16.34936)


## Pivot Wider by months

Since our ground-truth data has an annual resolution, we need to pivot our monthly features wider so that each observation contains all of the features for the full year. The main goal in this next step is to aggregate the features data into years while preserving the monthly information as separate columns. To achieve this, the code performs the following steps:

1. Separate the 'geometry' column temporarily to avoid duplicating it during unstacking.
2. Create a multi-level index using 'lon', 'lat', 'year', and 'month' in the DataFrame.
3. Unstack the DataFrame, transforming the innermost index level ('month') into new columns, creating a wide format DataFrame where each row represents a unique combination of 'lon', 'lat', and 'year', and the columns contain the values of the original features for each month.
4. Merge the 'geometry' column back into the DataFrame.

The resulting DataFrame contains the features aggregated by year with separate columns for each month's data. The naming convention will be `featurenumber_monthnumber`, so `587_12` corresponds to the 587th feature generated in December for that year 

In [9]:
# Store the 'geometry' column separately before unstacking
geometry_col = features_new_gdf[['lon', 'lat', 'geometry']].drop_duplicates(subset=['lon', 'lat'])

# Perform the unstacking operation without the 'geometry' column
features_gs_no_geometry = features_gs.drop(columns=['geometry'])
features = features_gs_no_geometry.set_index(['lon', 'lat', 'year', 'month']).unstack()
features.columns = features.columns.map(lambda x: '{}_{}'.format(*x))

# Merge the 'geometry' column back into the features DataFrame
features = features.reset_index().merge(geometry_col, on=['lon', 'lat'])


In [10]:

# Convert the 'features' DataFrame to a GeoDataFrame
features_gdf = gpd.GeoDataFrame(features, geometry=features['geometry'], crs='EPSG:4326')

In [11]:
features_gdf
# Calculate the total number of NaN cells
total_na_cells = features_gdf.isna().sum().sum()
print(f"Total NaN cells: {total_na_cells}")

Total NaN cells: 237011000


## Join features to ground data

In [52]:
# Now lets combine the sea data 
spatial_join = gpd.sjoin(features_gdf, sea_unq_join, how='right', predicate = 'within')

In [53]:
spatial_join.head()

,index_left,lon,lat,year,0_1,0_2,0_3,0_4,0_5,0_6,...,999_5,999_6,999_7,999_8,999_9,999_10,999_11,999_12,sea_unq,geometry
0,40939.0,27.81466,-13.669357,2020.0,NaN,0.0,NaN,0.0,0.0,0.0,...,0.474246,0.417571,0.135569,0.003355,0.004876,0.003185,0.187867,0.156783,1,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
0,40936.0,27.81466,-13.669357,2017.0,NaN,NaN,0.0,NaN,0.0,0.0,...,0.532948,0.469076,0.007786,0.006779,0.004811,0.001675,0.029891,NaN,1,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
0,40940.0,27.81466,-13.669357,2021.0,NaN,NaN,NaN,0.0,0.0,0.0,...,0.645601,0.673485,0.631725,0.004959,0.001359,0.001627,0.052683,NaN,1,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
0,40941.0,27.81466,-13.669357,2022.0,NaN,NaN,NaN,NaN,0.0,0.0,...,0.411212,0.286194,0.004574,0.006873,0.003651,0.087053,0.001866,0.001954,1,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
0,40934.0,27.81466,-13.669357,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.037791,NaN,NaN,NaN,1,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."


In [54]:
features_join = spatial_join.merge(country_sea, on=['year', 'sea_unq'], how='inner')

In [55]:
# Drop the redundant independent lon and lat columns because now that they are in a separate geometry column
features_join = features_join.drop(['geometry_x', 'index_left'], axis = 1)
    # Store the geometry column separately
geometry_col = features_join['geometry_y']
    # Remove the geometry column from the DataFrame
features_join = features_join.drop(columns=['geometry_y'])

In [56]:

features_join = features_join.replace([np.inf, -np.inf], np.nan)
# Remove the geometry column from the DataFrame
features_join = features_join.set_index(['year', 'sea_unq'])
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit_transform(features_join)
features_join[:] = imputer.transform(features_join)
features_join = features_join.reset_index()
# Add the geometry column back to the DataFrame


In [57]:
features_join = pd.concat([features_join, geometry_col], axis=1)
features_join = features_join.drop(['lat', 'lon'], axis = 1)


In [58]:
features_join['geometry'] = geometry_col
grouped_features = features_join.groupby(['year', 'sea_unq']).mean()


/tmp/ipykernel_2493574/3366700449.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_features = features_join.groupby(['year', 'sea_unq']).mean()


In [61]:
grouped_features

0_1       0_2       0_3       0_4       0_5       0_6  \
year   sea_unq                                                               
2016.0 1        0.000000  0.000863  0.000783  0.000000  0.000000  0.000000   
       2        0.000069  0.000863  0.000783  0.000000  0.000002  0.000014   
       7        0.001141  0.000863  0.000783  0.000329  0.000000  0.000000   
       9        0.001131  0.000863  0.000783  0.000006  0.000004  0.000010   
       10       0.001131  0.000863  0.000783  0.000000  0.000000  0.000000   
...                  ...       ...       ...       ...       ...       ...   
2021.0 388      0.001131  0.000076  0.000106  0.000075  0.000046  0.000112   
       389      0.001131  0.000863  0.000018  0.000104  0.000202  0.000424   
       390      0.000821  0.000345  0.000180  0.000227  0.000306  0.000457   
       391      0.001131  0.000863  0.000353  0.000323  0.000244  0.000222   
       392      0.001661  0.001136  0.000874  0.000720  0.000429  0.000714   

                     0_7           0_8       0_9      0_10  ...  prop_mix  \
year   sea_unq                                              ...             
2016.0 1        0.000000  6.157999e-06  0.000207  0.001568  ...  0.000000   
       2        0.000047  6.299240e-05  0.000168  0.001568  ...  0.000000   
       7        0.000000  1.008277e-03  0.001360  0.002211  ...  0.181102   
       9        0.000014  2.590917e-05  0.000110  0.001568  ...  0.069018   
       10       0.000000  3.113844e-07  0.000012  0.001568  ...  0.000000   
...                  ...           ...       ...       ...  ...       ...   
2021.0 388      0.000144  8.372727e-04  0.001012  0.001649  ...  0.000000   
       389      0.000395  7.513932e-04  0.001059  0.002266  ...  0.000000   
       390      0.000474  9.321970e-04  0.001530  0.002215  ...  0.000000   
       391      0.000311  1.540414e-03  0.001683  0.002434  ...  0.000000   
       392      0.001367  2.518747e-03  0.003055  0.004092  ...  0.000000   

                log_maize  log_sweetpotatoes  log_groundnuts  log_soybeans  \
year   sea_unq                                                               
2016.0 1         4.058626           6.364023        5.935403      6.565149   
       2         3.179960           6.364023        5.935403      6.565149   
       7         3.387211           0.689155        5.935403      6.565149   
       9         2.703935           6.364023       -1.408767      6.565149   
       10        3.714757           2.525729        3.354421      6.565149   
...                   ...                ...             ...           ...   
2021.0 388       7.939201           9.367183        8.098897      7.336848   
       389       7.844633           6.364023        8.048788      6.565149   
       390       7.792168           7.863267        8.154788      6.565149   
       391       7.652396           6.364023        8.065208      6.565149   
       392       7.641359           6.364023        7.960925      6.565149   

                loss_ind  drought_loss_ind  flood_loss_ind  animal_loss_ind  \
year   sea_unq                                                                
2016.0 1             0.0               0.0             0.0              0.0   
       2             0.0               0.0             0.0              0.0   
       7             1.0               1.0             0.0              0.0   
       9             1.0               0.0             0.0              0.0   
       10            1.0               0.0             0.0              0.0   
...                  ...               ...             ...              ...   
2021.0 388           0.0               0.0             0.0              0.0   
       389           1.0               1.0             0.0              0.0   
       390           1.0               0.0             0.0              0.0   
       391           1.0               0.0             1.0              0.0   
       392           1.0               0.

In [67]:
grouped_features.to_csv('/capstone/mosaiks/repos/preprocessing/data/grouped_features.csv', index=True)